# Gait analysis

In [ ]:
import importlib.resources 
import pickle
from pathlib import Path

from paradigma.constants import DataColumns
from paradigma.config import IMUConfig, GaitConfig
from paradigma.gait.gait_analysis import extract_gait_features, detect_gait, \
    extract_arm_activity_features, filter_gait, quantify_arm_swing, aggregate_arm_swing_params
from paradigma.preprocessing import preprocess_imu_data
from paradigma.util import load_tsdf_dataframe

In [ ]:
branch = 'gait'
sensor = 'imu'

gait_detection_classifier_package_filename = 'gait_detection_clf_package.pkl'
gait_filtering_classifier_package_filename = 'gait_filtering_clf_package.pkl'

path_to_data =  Path('../../../tests/data')
path_to_assets = Path('../../../src/paradigma/assets')

path_to_sensor_data = path_to_data / '1.prepared_data' / sensor

# Cell has the tag 'parameters', so it won't overwrite the parameters when running the notebook in tests

# Load data

In [ ]:
df, metadata_time, metadata_values = load_tsdf_dataframe(path_to_sensor_data, prefix='IMU')

## Preprocessing

In [ ]:
config = IMUConfig()

df_preprocessed = preprocess_imu_data(
    df=df,
    config=config, 
    sensor='both'
)

## Extract gait features

In [ ]:
config = GaitConfig(step='gait')

df_gait = extract_gait_features(
    df=df_preprocessed,
    config=config
)

## Detect gait

In [ ]:
full_path_to_classifier_package = path_to_assets / gait_detection_classifier_package_filename

with importlib.resources.files('paradigma.assets').joinpath('gait_detection_clf_package.pkl').open('rb') as f:
    clf_package = pickle.load(f)
 
df_gait[DataColumns.PRED_GAIT_PROBA] = detect_gait(
    df=df_gait,
    clf_package=clf_package
)

## Extract arm actvity features

In [ ]:
config = GaitConfig(step='arm_activity')

df_arm_activity = extract_arm_activity_features(
    config=config,
    df_timestamps=df_preprocessed,
    df_predictions=df_gait,
    threshold=clf_package.threshold
)

## Filter gait

In [ ]:
full_path_to_classifier_package = path_to_assets / gait_filtering_classifier_package_filename

with importlib.resources.files('paradigma.assets').joinpath('gait_filtering_clf_package.pkl').open('rb') as f:
    clf_package = pickle.load(f)

df_arm_activity[DataColumns.PRED_NO_OTHER_ARM_ACTIVITY_PROBA] = filter_gait(
    df=df_arm_activity,
    clf_package=clf_package
)

## Quantify arm swing

In [ ]:
config = GaitConfig(step='arm_activity')

df_quantified_unfiltered, df_quantified_filtered, segment_meta = quantify_arm_swing(
    df_timestamps=df_preprocessed,
    df_predictions=df_arm_activity,
    classification_threshold=clf_package.threshold,
    window_length_s=config.window_length_s,
    max_segment_gap_s=config.max_segment_gap_s,
    min_segment_length_s=config.min_segment_length_s,
    fs=config.sampling_frequency
)

## Aggregate quantification

In [ ]:
arm_swing_params = aggregate_arm_swing_params(
    df_arm_swing_params=df_quantified_filtered,
    segment_meta=segment_meta,
    aggregates=['median', '95p']
)